In [1]:
! pip install -r requirementsColab.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 18.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl 

In [2]:
from transformers import  AutoModelForSequenceClassification, AutoTokenizer

# Define pre-trained model
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"

# Get the classifier
classifier = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

# Get the tokenizer
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [3]:
data = "data/spam_ham_dataset.csv"

In [4]:
import pandas as pd

df = pd.read_csv(data)

In [5]:
# Removing Unnecessary column
df.drop('Unnamed: 0', axis=1, inplace = True)

In [6]:
df.head()

,label,text,label_num
0,ham,Subject: enron methanol ; meter # : 988291\r\n...,0
1,ham,"Subject: hpl nom for january 9 , 2001\r\n( see...",0
2,ham,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0
3,spam,"Subject: photoshop , windows , office . cheap ...",1
4,ham,Subject: re : indian springs\r\nthis deal is t...,0


In [7]:
# Check the data overall
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5171 entries, 0 to 5170
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   label      5171 non-null   object
 1   text       5171 non-null   object
 2   label_num  5171 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 121.3+ KB


In [8]:
# To make this simple we will label and only keep text label_num
df = df.loc[:,["text", "label_num"]]

In [9]:
df.head()

,text,label_num
0,Subject: enron methanol ; meter # : 988291\r\n...,0
1,"Subject: hpl nom for january 9 , 2001\r\n( see...",0
2,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0
3,"Subject: photoshop , windows , office . cheap ...",1
4,Subject: re : indian springs\r\nthis deal is t...,0


In [10]:
# Split the data into train and evaluation (stratified)
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df, train_size=0.8,stratify=df.label_num, random_state=42)

In [11]:
df_train.shape

(4136, 2)

In [12]:
df_test.shape

(1035, 2)

In [13]:
from datasets import Dataset, DatasetDict
datasets = DatasetDict({
    "train": Dataset.from_pandas(df_train),
    "test": Dataset.from_pandas(df_test)
})

In [14]:
datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'label_num', '__index_level_0__'],
        num_rows: 4136
    })
    test: Dataset({
        features: ['text', 'label_num', '__index_level_0__'],
        num_rows: 1035
    })
})

In [15]:
# Check the datasets
print("Dataset Dict:\n", datasets)
print("\n\nTrain's features:\n", datasets["train"].features)
print("\n\nFirst row of Train:\n", datasets["train"][0])

Dataset Dict:
 DatasetDict({
    train: Dataset({
        features: ['text', 'label_num', '__index_level_0__'],
        num_rows: 4136
    })
    test: Dataset({
        features: ['text', 'label_num', '__index_level_0__'],
        num_rows: 1035
    })
})


Train's features:
 {'text': Value(dtype='string', id=None), 'label_num': Value(dtype='int64', id=None), '__index_level_0__': Value(dtype='int64', id=None)}


First row of Train:
 {'text': "Subject: re : the hstoett lady sucklng huge ccok\r\nwhat ' s your pleasure , squire ?\r\nkuwo\r\n", 'label_num': 1, '__index_level_0__': 656}


In [16]:
# Tokenize the text, and truncate the text if it exceed the tokenizer maximum length. Batched=True to tokenize multiple texts at the same time.
tokenized_datasets = datasets.map(lambda dataset: tokenizer(dataset['text'], truncation=True), batched=True)

print(tokenized_datasets)

Map:   0%|          | 0/4136 [00:00<?, ? examples/s]

Map:   0%|          | 0/1035 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label_num', '__index_level_0__', 'input_ids', 'attention_mask'],
        num_rows: 4136
    })
    test: Dataset({
        features: ['text', 'label_num', '__index_level_0__', 'input_ids', 'attention_mask'],
        num_rows: 1035
    })
})


In [17]:
# Check the first row
print(tokenized_datasets["train"][0])

{'text': "Subject: re : the hstoett lady sucklng huge ccok\r\nwhat ' s your pleasure , squire ?\r\nkuwo\r\n", 'label_num': 1, '__index_level_0__': 656, 'input_ids': [101, 3395, 1024, 2128, 1024, 1996, 26236, 3406, 6582, 3203, 11891, 19666, 2290, 4121, 10507, 6559, 2054, 1005, 1055, 2115, 5165, 1010, 21263, 1029, 13970, 12155, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [18]:
tokenized_datasets = tokenized_datasets.remove_columns(["text", "__index_level_0__"])
tokenized_datasets = tokenized_datasets.rename_column("label_num", "labels")

In [19]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 4136
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 1035
    })
})

In [20]:
from transformers import DataCollatorWithPadding, TrainingArguments, Trainer
import numpy as np
import evaluate

# Padding for batch of data that will be fed into model for training
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Training args
training_args = TrainingArguments("test-trainer",
                                  num_train_epochs=3,
                                  evaluation_strategy="epoch",
                                  weight_decay=5e-4,
                                  save_strategy="no",
                                  report_to="none")

In [21]:
# Metric for validation error
def compute_metrics(eval_preds):
    metric = evaluate.load("glue", "mrpc") # F1 and Accuracy
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [22]:
# Define trainer
trainer = Trainer(
    classifier,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [23]:
# Start the fine-tuning
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.154500,0.039801,0.988406,0.979798
2,0.016600,0.036213,0.991304,0.984772
3,0.000700,0.023456,0.993237,0.988314


TrainOutput(global_step=1551, training_loss=0.055379932783970395, metrics={'train_runtime': 598.4749, 'train_samples_per_second': 20.733, 'train_steps_per_second': 2.592, 'total_flos': 1500988094171136.0, 'train_loss': 0.055379932783970395, 'epoch': 3.0})

In [24]:
from sklearn.metrics import classification_report

# Make prediction on evaluation dataset
y_pred = trainer.predict(tokenized_datasets["test"]).predictions
y_pred = np.argmax(y_pred, axis=-1)

# Get the true labels
y_true = tokenized_datasets["test"]["labels"]
y_true = np.array(y_true)

# Print the classification report
print(classification_report(y_true, y_pred, digits=3))

              precision    recall  f1-score   support

           0      0.995     0.996     0.995       735
           1      0.990     0.987     0.988       300

    accuracy                          0.993      1035
   macro avg      0.992     0.991     0.992      1035
weighted avg      0.993     0.993     0.993      1035



In [25]:
trainer.save_model('model/senti_model')

In [26]:
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer

def classify_email(email_text, model_dir):
    # Load the saved model
    model = DistilBertForSequenceClassification.from_pretrained(model_dir)

    # Load the tokenizer
    tokenizer = DistilBertTokenizer.from_pretrained(checkpoint)

    # Tokenize the email text
    inputs = tokenizer(email_text, return_tensors="pt")

    # Make predictions
    outputs = model(**inputs)

    # Interpret the output
    predicted_label = "spam" if outputs.logits[0][1] > outputs.logits[0][0] else "ham"

    return predicted_label


In [28]:
#email_text = "This is a legitimate email regarding your recent purchase."
email_text = "Subject: hpl nom for january 9 , 2001 ( see attached file : hplnol 09 . xls ) - hplnol 09 . xls"
model_dir = "model/senti_model"

prediction = classify_email(email_text, model_dir)
print("Predicted Label:", prediction)


Predicted Label: ham


In [43]:
trainer.save_model('distilbert-base-uncased-finetuned-sst-2-english')

In [44]:
! ls -ltr

total 241332
drwxr-xr-x 1 root root      4096 May 17 13:37 sample_data
-rw-r--r-- 1 root root        49 May 21 00:36 requirementsColab.txt
drwxr-xr-x 2 root root      4096 May 21 00:36 data
drwxr-xr-x 2 root root      4096 May 21 00:39 test-trainer
drwxr-xr-x 3 root root      4096 May 21 00:49 model
-rw-r--r-- 1 root root 247099210 May 21 00:51 senti_model.zip
drwxr-xr-x 2 root root      4096 May 21 01:37 distilbert-base-uncased-finetuned-sst-2-english


In [38]:
from huggingface_hub import notebook_login

In [46]:
notebook_login()

In [41]:
from huggingface_hub import HfApi

api = HfApi()

In [47]:
api.upload_folder(
    folder_path="./distilbert-base-uncased-finetuned-sst-2-english",
    path_in_repo = ".",
    repo_id="shraman5702/distilbert-base-uncased-finetuned-sst-2-english",
    repo_type="model"
)

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/4.98k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/shraman5702/distilbert-base-uncased-finetuned-sst-2-english/commit/f5f7d91d6ffcc14b2e76bfe13ef72c3166265028', commit_message='Upload folder using huggingface_hub', commit_description='', oid='f5f7d91d6ffcc14b2e76bfe13ef72c3166265028', pr_url=None, pr_revision=None, pr_num=None)